In [1]:
# As requested from Bruce, devided by the unsubsription

import pandas as pd
import numpy as np
import datetime
import os
import gc
import glob
import sys
samplerows=None
pd.options.display.max_columns=111
os.getcwd()

import logging
logging.basicConfig(filename='BL_LiveRampb_1_to_1_2020Q3_subscribe.log', level=logging.INFO)
logging.info('Started: '+str(datetime.datetime.now()))

In [2]:
startdate = '2020-08-02'
enddate = '2020-10-31'

lr_impr_cols=['Customer_Link','Event Time', 'Advertiser ID', 'Campaign ID', 'Ad ID', 'Rendering ID','Creative Version','Site ID (DCM)','Placement ID','Impression ID']
lr_click_cols=['Customer_Link','Event Time', 'Advertiser ID', 'Campaign ID', 'Ad ID', 'Rendering ID','Creative Version','Site ID (DCM)','Placement ID','Impression ID']
lr_act_cols=['Customer_Link','Event Time', 'Advertiser ID', 'Campaign ID', 'Ad ID', 'Rendering ID','Creative Version','Site ID (DCM)','Placement ID','Activity ID']



In [3]:
df_impr_lr_returned=pd.read_csv('./Aggregated_files_to_use/LR/BL_aggregate_impression_LR_returned_Q3_JL_2020-11-29.tsv.gz',
                            dtype=str,sep="\t",usecols=lr_impr_cols)
df_click_lr_returned=pd.read_csv('./Aggregated_files_to_use/LR/BL_aggregate_click_LR_returned_Q3_JL_2020-11-29.tsv.gz',
                            dtype=str,sep="\t",usecols=lr_click_cols)

df_act_lr_returned=pd.read_csv('./Aggregated_files_to_use/LR/BL_aggregate_activity_LR_returned_Q3_JL_2020-11-29.tsv.gz',
                            dtype=str,sep="\t",usecols=lr_act_cols)
gc.collect()

0

In [4]:
print(df_impr_lr_returned.shape,df_impr_lr_returned.drop_duplicates().shape)
print(df_click_lr_returned.shape,df_click_lr_returned.drop_duplicates().shape)
print(df_act_lr_returned.shape,df_act_lr_returned.drop_duplicates().shape)

(41299628, 10) (41299628, 10)
(784220, 10) (784220, 10)
(7519539, 10) (7519539, 10)


In [5]:
# Cheched about the missing part and it looks good
print(datetime.datetime.now())

2020-12-22 15:36:47.897978


In [6]:
dict_match_rate_overall={}
dict_match_rate_overall.update({"impression_match_rate":df_impr_lr_returned[df_impr_lr_returned['Customer_Link']!="UNMATCHED"].shape[0]/df_impr_lr_returned.shape[0]})
dict_match_rate_overall.update({"click_match_rate":df_click_lr_returned[df_click_lr_returned['Customer_Link']!="UNMATCHED"].shape[0]/df_click_lr_returned.shape[0]})
dict_match_rate_overall.update({"activity_match_rate":df_act_lr_returned[df_act_lr_returned['Customer_Link']!="UNMATCHED"].shape[0]/df_act_lr_returned.shape[0]})
dict_match_rate_overall

{'impression_match_rate': 0.24690776875762657,
 'click_match_rate': 0.30110810741883653,
 'activity_match_rate': 0.4904053293692605}

# Site

In [7]:
all_2020Q3_site_matching_files=glob.glob("/mnt/drv5/dcm_logs/match_tables/*site*")
all_2020Q3_site_matching_files=[x for x in all_2020Q3_site_matching_files if x.split("dcm_account7252_match_table_sites_")[1][:8]>=startdate.replace("-","")]
all_2020Q3_site_matching_files=[x for x in all_2020Q3_site_matching_files if x.split("dcm_account7252_match_table_sites_")[1][:8]<=enddate.replace("-","")]
len(all_2020Q3_site_matching_files)


91

In [8]:
all_2020Q3_site_matching_files[0]

'/mnt/drv5/dcm_logs/match_tables/dcm_account7252_match_table_sites_20200830_20200831_030201_3033490353.csv.gz'

In [9]:
site_name_2020Q3=pd.DataFrame()
for file in all_2020Q3_site_matching_files:
    df=pd.read_csv(file,dtype=str,compression="gzip")
    site_name_2020Q3=site_name_2020Q3.append(df)
site_name_2020Q3=site_name_2020Q3.drop_duplicates()
site_name_2020Q3=site_name_2020Q3[['Site ID (DCM)','Site (DCM)']].drop_duplicates()
site_name_2020Q3.shape

(791, 2)

In [10]:

site_name_2020Q3.groupby("Site ID (DCM)")['Site (DCM)'].count().to_frame().reset_index().sort_values("Site (DCM)",ascending=False)
# site_name_2020Q3[site_name_2020Q3['Site ID (DCM)']=="5056339"]

# All unique 1 to 1 mapping between the site id and site name


,Site ID (DCM),Site (DCM)
0,1232325,1
520,1987432,1
522,1987434,1
523,1987435,1
524,1996992,1
...,...,...
266,1377555,1
267,1377556,1
268,1377717,1
269,1377848,1


In [30]:
site_name_2020Q3[site_name_2020Q3['Site ID (DCM)']=="5056339"]

,Site ID (DCM),Site (DCM)
745,5056339,Twitter - Unofficial


In [11]:
print(df_impr_lr_returned.shape)

impr_site=pd.merge(df_impr_lr_returned,site_name_2020Q3,on="Site ID (DCM)").groupby("Site (DCM)")['Event Time'].count().to_frame().reset_index()
impr_site=impr_site.rename(columns={"Event Time":"impression_rows"})
print(impr_site['impression_rows'].sum())
impr_site

(41299628, 10)
41299628


,Site (DCM),impression_rows
0,The Trade Desk,41298834
1,YouTube.com,794


In [12]:
print(df_click_lr_returned.shape)

click_site=pd.merge(df_click_lr_returned,site_name_2020Q3,on="Site ID (DCM)").groupby("Site (DCM)")['Event Time'].count().to_frame().reset_index()
click_site=click_site.rename(columns={"Event Time":"click_rows"})
print(click_site['click_rows'].sum())
click_site

(784220, 10)
784220


,Site (DCM),click_rows
0,Facebook,652376
1,"Facebook Inc – Instagram, LLC",12977
2,Pinterest Inc.,7818
3,The Trade Desk,110972
4,YouTube.com,77


In [13]:
print(df_act_lr_returned.shape)

activity_site=pd.merge(df_act_lr_returned,site_name_2020Q3,on="Site ID (DCM)").groupby("Site (DCM)")['Event Time'].count().to_frame().reset_index()
activity_site=activity_site.rename(columns={"Event Time":"activity_rows"})
print(activity_site['activity_rows'].sum())
activity_site

(7519539, 10)
7519539


,Site (DCM),activity_rows
0,Facebook,5646762
1,"Facebook Inc – Instagram, LLC",75636
2,Pinterest Inc.,27907
3,The Trade Desk,1768791
4,YouTube.com,443


In [14]:
del df_impr_lr_returned
del df_click_lr_returned
del df_act_lr_returned
gc.collect()

20

# 1 to 1 analysis

In [15]:
# Regular audience only

# old_output_path='/mnt/drv5/jian/Big_Lots/BL_1_to_1_2020Q3/output/'
outputpath='./output_regular_only_'+str(datetime.datetime.now().date())+"/"
try:
    os.stat(outputpath)
except:
    os.mkdir(outputpath)
    

In [16]:
samplerows = None

Q3_date_str_pairs=[startdate,enddate]
print(Q3_date_str_pairs)

Q3_2020_start=int(startdate.replace("-",""))
Q3_2020_end=int(enddate.replace("-",""))
Q3_date_int_pairs=[Q3_2020_start,Q3_2020_end]
print(Q3_date_int_pairs)


['2020-08-02', '2020-10-31']
[20200802, 20201031]


In [17]:
idlist=pd.read_table("./Q3_mapping_file/CL_BigLots_BL_mapping_file_2020Q3_JL_2020-11-24_20201130_160714_0000.psv.gz",
                          dtype=str,compression="gzip",sep="|",usecols=['customer_id_hashed','Customer_Link'])
print(idlist.shape)
idlist=idlist[idlist['Customer_Link']!="UNMATCHED"]
idlist = idlist.drop_duplicates(['Customer_Link'])
idlist = idlist.drop_duplicates(['customer_id_hashed'])
print(idlist.shape)
idlist.head(2)

(32396452, 2)
(31150779, 2)


,Customer_Link,customer_id_hashed
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...


# All_placement

In [18]:
match_table_folder='/mnt/drv5/dcm_logs/match_tables/'

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
mapping_files=list(recursive_file_gen(match_table_folder))

In [19]:
campaign_files=[x for x in mapping_files if "campaigns" in x]
df_campaign=pd.DataFrame({"file_path":campaign_files})
df_campaign['start_date']=df_campaign['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[5]))
df_campaign['end_date']=df_campaign['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[6]))

placements_files=[x for x in mapping_files if "placements" in x] # placements only
df_placements=pd.DataFrame({"file_path":placements_files})
df_placements['start_date']=df_placements['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[5]))
df_placements['end_date']=df_placements['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[6]))

In [20]:
campaigns_df=df_campaign[(df_campaign['end_date']>=Q3_date_int_pairs[0]) & (df_campaign['start_date']<=Q3_date_int_pairs[1])]
placements_df=df_placements[(df_placements['end_date']>=Q3_date_int_pairs[0]) & (df_placements['start_date']<=Q3_date_int_pairs[1])]



In [21]:
campaign_data_df=pd.DataFrame()
for campaign_file in campaigns_df['file_path'].tolist():
    df=pd.read_csv(campaign_file,dtype=str,compression="gzip")
    df=df[df['Advertiser ID']=='8095847']
    campaign_data_df=campaign_data_df.append(df)
Quarter_campaign_id_list=campaign_data_df['Campaign ID'].tolist()

In [22]:
placements_data_df=pd.DataFrame()
for placements_file in placements_df['file_path'].tolist():
    df=pd.read_csv(placements_file,dtype=str,compression="gzip")
    df=df[df['Campaign ID'].isin(Quarter_campaign_id_list)]
    placements_data_df=placements_data_df.append(df)
    
placements_data_df=placements_data_df.reset_index()
del placements_data_df['index']

print(placements_data_df.shape)

(4469035, 15)


In [23]:
placements_LiveRamp_data_df=placements_data_df[placements_data_df['Placement'].apply(lambda x: "rewards" in x.lower())]
print(placements_LiveRamp_data_df.shape)

(2074529, 15)


In [24]:
extracted_LR_Placement_set=set(placements_LiveRamp_data_df["Placement ID"].unique().tolist())
len(extracted_LR_Placement_set)

23946

In [25]:
placements_LiveRamp_data_df.to_csv("./BL_2020Q3_LiveRamp_Placement_JL_"+str(datetime.datetime.now().date())+".csv",index=False)


In [26]:
Q3_2020_start,Q3_2020_end

(20200802, 20201031)

In [27]:
str(Q3_2020_start)

'20200802'

# Sales

In [ ]:
def convert_dig_tue_to_sat(str_digit_date_Tue):
    date_input=datetime.datetime.strptime(str_digit_date_Tue,"%Y%m%d").date()
    weekday=date_input.weekday()
    if weekday>3:
        print("the date in the file name str is not between Mon and Thur")
        return np.nan
    else:
        date_output=date_input-datetime.timedelta(days=weekday+2)
        return date_output


In [ ]:
daily_sales_list=glob.glob("/mnt/drv5/lr_biglots_data/BL_all_item_level_POS_data/*.txt")
df_daily_sales_list=pd.DataFrame({"file_path":daily_sales_list})
df_daily_sales_list['week_end_dt']=df_daily_sales_list['file_path'].apply(lambda x: x.split("MediaStormDailySales")[1][:8])
df_daily_sales_list=df_daily_sales_list[df_daily_sales_list['week_end_dt'].str.isdigit()]
df_daily_sales_list['week_end_dt']=df_daily_sales_list['week_end_dt'].apply(lambda x: str(convert_dig_tue_to_sat(x)).replace("-",""))
df_daily_sales_list=df_daily_sales_list[((df_daily_sales_list['week_end_dt']>=str(Q3_2020_start)) & (df_daily_sales_list['week_end_dt']<=str(Q3_2020_end)))]

print(df_daily_sales_list['week_end_dt'].min(),df_daily_sales_list['week_end_dt'].max())
daily_sales_list=df_daily_sales_list['file_path'].tolist()
daily_sales_list=sorted(daily_sales_list,key=lambda x: x.split("MediaStormDailySales")[1][:8])
print(len(daily_sales_list))


In [ ]:
df_prod_taxonomy=pd.read_table("./prod_taxonomy/MediaStormProductTaxonomy20201101-135215-182.txt",dtype=str,sep="|")
df_prod_taxonomy=df_prod_taxonomy[['class_code_id','subclass_id','department_id','division_id']].drop_duplicates()

df_department_name=pd.read_table("/mnt/drv5/jian/Big_Lots/BL_1_to_1_2019Q4/prod_taxonomy/MediaStorm Data Extract - Department Names.txt",sep="|",dtype=str)
df_department_name[df_department_name['department_desc'].str.contains("MATTRESS")]

In [ ]:
dftrans = pd.DataFrame()
dftrans_furniture_all=pd.DataFrame()
dftrans_furniture_type=pd.DataFrame()

for file_path in daily_sales_list:
    df = pd.read_csv(file_path,sep = '|',nrows = samplerows,dtype=str)
    if 'transaction_dt' in df.columns:
        df = df.rename(columns = {'transaction_dt':'transaction_date'})
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    df['location_id']=df['location_id'].astype(int)
    df=df[~pd.isnull(df['customer_id_hashed'])]
    
    df_all=df.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id'])[['item_transaction_units','item_transaction_amt']].apply(sum).reset_index()
    df_all=df_all.rename(columns={"item_transaction_amt":"total_transaction_amt","item_transaction_units":"total_transaction_units"})
    
    df_furniture=pd.merge(df,df_prod_taxonomy,on=['class_code_id','subclass_id'],how="left")
    df_furniture=df_furniture[df_furniture['division_id']=="9"]
    df_furniture_type=df_furniture.copy()

    df_furniture=df_furniture.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id'])[['item_transaction_units','item_transaction_amt']].apply(sum).reset_index()
    df_furniture=df_furniture.rename(columns={"item_transaction_amt":"total_furniture_sales","item_transaction_units":"total_furniture_units"})

    df_furniture_type['furtinture_type']=np.where(df_furniture_type['department_id']=="615","Matress_Furniture","Other_Furniture")
    df_furniture_type=df_furniture_type.groupby(['customer_id_hashed','transaction_date','transaction_id','location_id','furtinture_type'])[['item_transaction_units','item_transaction_amt']].apply(sum).reset_index()
    df_furniture_type=df_furniture_type.rename(columns={"item_transaction_amt":"type_furniture_sales","item_transaction_units":"type_furniture_units"})
    
    dftrans = dftrans.append(df_all,ignore_index = True)
    dftrans_furniture_all = dftrans_furniture_all.append(df_furniture,ignore_index = True)
    dftrans_furniture_type = dftrans_furniture_type.append(df_furniture_type,ignore_index = True)
    print(file_path,datetime.datetime.now())

del df
del df_all
del df_furniture
del df_furniture_type

gc.collect()

In [ ]:
dfsummary = dftrans.groupby('transaction_date')['total_transaction_amt'].sum().to_frame().reset_index()
dfsummary.to_csv(outputpath + 'totaltransactionsbyday.csv')

In [ ]:
dfsummary.head(2)

In [ ]:
outputpath

In [ ]:
dfsummary['transaction_date'].min(),dfsummary['transaction_date'].max(),dfsummary['transaction_date'].nunique()

In [ ]:
dftrans['total_transaction_amt'].sum(),dftrans_furniture_all['total_furniture_sales'].sum(),dftrans_furniture_type['type_furniture_sales'].sum()



In [ ]:
dftrans.shape

In [ ]:
dftrans_furniture_type.shape,dftrans_furniture_all.shape

In [ ]:
dftrans_furniture_type.head(5)

In [ ]:
dftrans_furniture_type['furtinture_type'].unique()

In [ ]:
dftrans_furniture_all.head(2)

# Overall Sales

In [ ]:
dftrans['instore_transactions_units'] = np.where(dftrans['location_id']!=6990,
                                      dftrans['total_transaction_units'],0)
dftrans['instore_revenue'] = np.where(dftrans['location_id']!=6990,
                                      dftrans['total_transaction_amt'],0)
dftrans['online_transactions_units'] = np.where(dftrans['location_id']==6990,
                                      dftrans['total_transaction_units'],0)
dftrans['online_revenue'] = np.where(dftrans['location_id']==6990,
                                      dftrans['total_transaction_amt'],0)
dftrans['total_transactions'] = 1
dftrans['instore_transactions'] = np.where(dftrans['location_id']!=6990,
                                      1,0)
dftrans['online_transactions'] = np.where(dftrans['location_id']==6990,
                                      1,0)

dftrans_sum = dftrans.groupby('customer_id_hashed')['online_transactions_units', 'online_transactions','online_revenue',
                 'instore_transactions_units','instore_transactions','instore_revenue',
                 'total_transaction_units','total_transactions','total_transaction_amt'].sum().reset_index()

del dftrans
gc.collect()
dftrans_sum.head(2)

In [ ]:
dftrans_furniture_all['div9_instore_units'] = np.where(dftrans_furniture_all['location_id']!=6990,
                                      dftrans_furniture_all['total_furniture_units'],0)
dftrans_furniture_all['div9_instore_revenue'] = np.where(dftrans_furniture_all['location_id']!=6990,
                                      dftrans_furniture_all['total_furniture_sales'],0)
dftrans_furniture_all['div9_online_units'] = np.where(dftrans_furniture_all['location_id']==6990,
                                      dftrans_furniture_all['total_furniture_units'],0)
dftrans_furniture_all['div9_online_revenue'] = np.where(dftrans_furniture_all['location_id']==6990,
                                      dftrans_furniture_all['total_furniture_sales'],0)
dftrans_furniture_all['div9_transactions'] = 1

dftrans_furniture_all['div9_instore_trans'] = np.where(dftrans_furniture_all['location_id']!=6990,
                                      1,0)
dftrans_furniture_all['div9_online_trans'] = np.where(dftrans_furniture_all['location_id']==6990,
                                      1,0)

dftrans_sum_furniture = dftrans_furniture_all.groupby('customer_id_hashed')['div9_online_units', 'div9_online_trans','div9_online_revenue',
                         'div9_instore_units','div9_instore_trans','div9_instore_revenue',
                         'total_furniture_units','div9_transactions','total_furniture_sales'].sum().reset_index()

del dftrans_furniture_all
gc.collect()
dftrans_sum_furniture.head(2)

In [ ]:
print(dftrans_furniture_type['furtinture_type'].unique())

In [ ]:
dftrans_sum_FurType=dftrans_furniture_type[['customer_id_hashed']].drop_duplicates()
for furtinture_type, df_group in dftrans_furniture_type.groupby("furtinture_type"):
    type_col_name_unit=furtinture_type+"_units"
    type_col_name_sales=furtinture_type+"_sales"
    
    df_group=df_group.rename(columns={"type_furniture_units":type_col_name_unit,"type_furniture_sales":type_col_name_sales})
    
    df_group[furtinture_type+'_instore_units'] = np.where(df_group['location_id']!=6990,
                                      df_group[type_col_name_unit],0)
    df_group[furtinture_type+'_instore_revenue'] = np.where(df_group['location_id']!=6990,
                                          df_group[type_col_name_sales],0)
    df_group[furtinture_type+'_online_units'] = np.where(df_group['location_id']==6990,
                                          df_group[type_col_name_unit],0)
    df_group[furtinture_type+'_online_revenue'] = np.where(df_group['location_id']==6990,
                                          df_group[type_col_name_sales],0)
    df_group[furtinture_type+'_transactions'] = 1

    df_group[furtinture_type+'_instore_trans'] = np.where(df_group['location_id']!=6990,
                                          1,0)
    df_group[furtinture_type+'_online_trans'] = np.where(df_group['location_id']==6990,
                                          1,0)

    df_group = df_group.groupby('customer_id_hashed')[furtinture_type+'_online_units',furtinture_type+'_online_trans',furtinture_type+'_online_revenue',
                         furtinture_type+'_instore_units',furtinture_type+'_instore_trans',furtinture_type+'_instore_revenue',
                         type_col_name_unit,furtinture_type+'_transactions',type_col_name_sales].sum().reset_index()

    dftrans_sum_FurType=pd.merge(dftrans_sum_FurType,df_group,on="customer_id_hashed",how="left")
dftrans_sum_FurType=dftrans_sum_FurType.fillna(0)
    

In [ ]:
del df_group
gc.collect()

In [ ]:
print(dftrans_sum.shape)
print(dftrans_furniture_type.shape)
print(dftrans_sum_FurType.shape)

In [ ]:
dftrans_sum=pd.merge(dftrans_sum,dftrans_sum_furniture,on="customer_id_hashed",how="left")
dftrans_sum=pd.merge(dftrans_sum,dftrans_sum_FurType,on="customer_id_hashed",how="left")
dftrans_sum=dftrans_sum.fillna(0)
del dftrans_furniture_type
del dftrans_sum_FurType
print(dftrans_sum.shape)


In [ ]:
idlist.head(2)

In [ ]:
idlist = pd.merge(idlist,dftrans_sum,on = ['customer_id_hashed'],how='left')
print("len(idlist.index): " + str(len(idlist.index)))
idlist.fillna(0,inplace = True)

del dftrans_sum
gc.collect()
idlist.head(2)

In [ ]:
print(len(extracted_LR_Placement_set))
print(datetime.datetime.now())

In [ ]:
gc.collect()


In [ ]:
# consider day time saving
daylist_saving_time_local=datetime.datetime(2020,3,8,1,0,0)
daylist_saving_time_utc=daylist_saving_time_local+datetime.timedelta(hours=5)
daylight_saving_int=datetime.datetime.timestamp(daylist_saving_time_utc)
daylight_saving_int=daylight_saving_int*10**6
print("daylight_saving_int",daylight_saving_int)

In [ ]:
os.listdir("./Aggregated_files_to_use/LR/")

In [ ]:
# Convert time zone

filename_impr="./Aggregated_files_to_use/LR/BL_aggregate_impression_LR_returned_Q3_JL_2020-11-29.tsv.gz"
chunksize = 10 ** 7
df_impr=pd.DataFrame()
for chunk in pd.read_csv(filename_impr, chunksize=chunksize,compression="gzip",dtype=str,sep="\t",nrows = samplerows):
    chunk=chunk[chunk['Advertiser ID']=='8095847']
    chunk=chunk[chunk['Customer_Link']!='UNMATCHED']
    chunk=chunk[chunk['Placement ID'].isin(extracted_LR_Placement_set)]
    df_impr=df_impr.append(chunk)

df_impr = df_impr.drop_duplicates()
del chunk
print("len_of_df_impr: "+str(len(df_impr)))


df_impr['event_timestamp2'] = df_impr['Event Time'].astype(int)

df_impr['event_timestamp2'] = np.where(df_impr['event_timestamp2']>=daylight_saving_int,
                                      df_impr['event_timestamp2']/1000-1000*60*60*4,
                                      df_impr['event_timestamp2']/1000-1000*60*60*5)

# df_impr['event_timestamp2']=df_impr['event_timestamp2']/1000-1000*60*60*5
# df_impr['event_timestamp2']=df_impr['event_timestamp2']/1000-1000*60*60*5
df_impr['event_timestamp2'] = pd.to_datetime(df_impr['event_timestamp2'], unit='ms')
df_impr['date'] = df_impr['event_timestamp2'].dt.date
df_impr = df_impr[df_impr['date']>=datetime.datetime.strptime(str(startdate), '%Y-%m-%d').date()]
df_impr = df_impr[df_impr['date']<=datetime.datetime.strptime(str(enddate), '%Y-%m-%d').date()]
df_impr.to_csv(outputpath + 'data_impression.csv')

###
dftest = df_impr.groupby('date').count()['event_timestamp2']
dftest.to_csv(outputpath + 'data_impressionbydate.csv')
###


df_impr['impression'] = 1

logging.info("df_impr['date'].unique():" + str(df_impr['date'].unique()))
dftest.head(2)

In [ ]:
df_impr = df_impr[['impression','Customer_Link']].groupby('Customer_Link').sum()
df_impr.reset_index(inplace = True)
df_impr['impressionunique'] = 1
idlist = pd.merge(idlist,df_impr,on = ['Customer_Link'],how='left')

logging.info("len(idlist.index): "+str(len(idlist.index))) 
print("len(idlist.index): "+str(len(idlist.index))) 

In [ ]:
del df_impr
gc.collect()

In [ ]:
####clicks

df_click = pd.read_table("./Aggregated_files_to_use/LR/BL_aggregate_click_LR_returned_Q3_JL_2020-11-29.tsv.gz",
                         compression="gzip",dtype=str,sep="\t",nrows = samplerows)

print("len_of_df_click: "+str(len(df_click)))
df_click = df_click[df_click['Advertiser ID']=='8095847']
df_click = df_click[df_click['Customer_Link']!='UNMATCHED']
df_click=df_click[df_click['Placement ID'].isin(extracted_LR_Placement_set)]
print("len_of_df_click: "+str(len(df_click)))

df_click = df_click.drop_duplicates()


df_click['event_timestamp2']=df_click['Event Time'].astype(int)

df_click['event_timestamp2'] = np.where(df_click['event_timestamp2']>=daylight_saving_int,
                                      df_click['event_timestamp2']/1000-1000*60*60*4,
                                      df_click['event_timestamp2']/1000-1000*60*60*5)

# df_click['event_timestamp2']=df_click['event_timestamp2']/1000-1000*60*60*5
# df_click['event_timestamp2']=df_click['event_timestamp2']/1000-1000*60*60*5
df_click['event_timestamp2'] = pd.to_datetime(df_click['event_timestamp2'], unit='ms')
df_click['date'] = df_click['event_timestamp2'].dt.date
df_click = df_click[df_click['date']>=datetime.datetime.strptime(str(startdate), '%Y-%m-%d').date()]
df_click = df_click[df_click['date']<=datetime.datetime.strptime(str(enddate), '%Y-%m-%d').date()]
df_click.to_csv(outputpath + 'data_click.csv')
###
dftest = df_click.groupby('date').count()['event_timestamp2']
dftest.to_csv(outputpath + 'data_clickbydate.csv')
###
logging.info("df_click['date'].unique()"+ str(df_click['date'].unique())) 
df_click['click'] = 1


In [ ]:
df_click = df_click[['click','Customer_Link']].groupby('Customer_Link').sum()
df_click.reset_index(inplace = True)
df_click['clickunique'] = 1
idlist = pd.merge(idlist,df_click,on = ['Customer_Link'],how='left')

logging.info("len(idlist.index)"+ str(len(idlist.index))) 
print("len(idlist.index): "+str(len(idlist.index))) 

In [ ]:
del df_click
gc.collect()

In [ ]:
####activity

filename_act="./Aggregated_files_to_use/LR/BL_aggregate_activity_LR_returned_Q3_JL_2020-11-29.tsv.gz"

chunksize = 10 ** 7
df_act=pd.DataFrame()
for chunk in pd.read_csv(filename_act, chunksize=chunksize,compression="gzip",dtype=str,sep="\t",nrows = samplerows):
    chunk=chunk[chunk['Advertiser ID']=='8095847']
    chunk=chunk[chunk['Customer_Link']!='UNMATCHED']
    chunk=chunk[chunk['Placement ID'].isin(extracted_LR_Placement_set)]
    df_act=df_act.append(chunk)

df_act = df_act.drop_duplicates()
del chunk
gc.collect()
df_act['event_timestamp2']=df_act['Event Time'].astype(int)

df_act['event_timestamp2'] = np.where(df_act['event_timestamp2']>=daylight_saving_int,
                                      df_act['event_timestamp2']/1000-1000*60*60*4,
                                      df_act['event_timestamp2']/1000-1000*60*60*5)

# df_act['event_timestamp2']=df_act['event_timestamp2']/1000-1000*60*60*5
# df_act['event_timestamp2']=df_act['event_timestamp2']/1000-1000*60*60*5
df_act['event_timestamp2'] = pd.to_datetime(df_act['event_timestamp2'], unit='ms')
df_act['date'] = df_act['event_timestamp2'].dt.date
df_act = df_act[df_act['date']>=datetime.datetime.strptime(str(startdate), '%Y-%m-%d').date()]
df_act = df_act[df_act['date']<=datetime.datetime.strptime(str(enddate), '%Y-%m-%d').date()]

df_act.to_csv(outputpath + 'data_activity.csv')

dftest = df_act.groupby('date').count()['event_timestamp2']
dftest.to_csv(outputpath + 'data_activitybydate.csv')

logging.info("df_act['date'].unique():"+str(df_act['date'].unique()))
df_act['activity'] = 1


In [ ]:
df_act = df_act[['activity','Customer_Link']].groupby('Customer_Link').sum()
df_act.reset_index(inplace = True)
df_act['activityunique'] = 1
idlist = pd.merge(idlist,df_act,on = ['Customer_Link'],how='left')

logging.info("len(idlist.index)" + str(len(idlist.index)))
print("len(idlist.index): "+str(len(idlist.index))) 

In [ ]:
del df_act
gc.collect()

In [ ]:
idlist.fillna(0,inplace = True)
idlist['IDs'] = 1
idlist.to_csv(outputpath + 'allactivity_id.csv',index = False)

In [ ]:
idlist.head(4)

In [ ]:
outputpath

In [1]:
import pandas as pd
import datetime
import numpy as np
import os
# outputpath='./output_regular_only_'+str(datetime.datetime.now().date())+"/"


In [12]:
# Email subscription division only
pd.options.display.max_columns=111

In [3]:
# Restart here if needed

outputpath='./output_regular_only_2020-12-05//'
idlist = pd.read_csv(outputpath + 'allactivity_id.csv')
# idlist['uniquetrans'] = np.where(idlist['total_transactions']>0,1,0)
# convert the col names
idlist=idlist.rename(columns={"total_furniture_units":"div9_units","total_furniture_sales":"div9_sales"})
idlist.head(2)


,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,...,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,impression,impressionunique,click,clickunique,activity,activityunique,IDs
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [13]:
idlist.head(2)


,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,impression,impressionunique,click,clickunique,activity,activityunique,IDs
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [14]:
idlist=idlist.rename(columns={"total_furniture_units":"div9_units","total_furniture_sales":"div9_sales"})
idlist.head(2)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,impression,impressionunique,click,clickunique,activity,activityunique,IDs
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [15]:
idlist.insert(idlist.columns.tolist().index("total_transaction_amt")+1,
              "unique_buyers",
              np.where(idlist['total_transactions']>0,1,0))
idlist.insert(idlist.columns.tolist().index("div9_sales")+1,
              "div9_buyers",
              np.where(idlist['div9_sales']>0,1,0))

idlist.insert(idlist.columns.tolist().index("Matress_Furniture_sales")+1,
              "Matress_Furniture_buyers",
              np.where(idlist['Matress_Furniture_sales']>0,1,0))
idlist.insert(idlist.columns.tolist().index("Other_Furniture_sales")+1,
              "Other_Furniture_buyers",
              np.where(idlist['Other_Furniture_sales']>0,1,0))

In [16]:
os.getcwd()

'/home/jian/analysis/Big_Lots/BL_1_to_1_2020Q3'

In [17]:
idlistwseg = pd.read_csv('./Q3_mapping_file/BL_mapping_file_2020Q3_JL_2020-11-24.csv',
                        usecols=['customer_id_hashed','segment']).drop_duplicates()
# The gross aggregated segments from a lot of small files, not the LR returned

print(idlistwseg.shape,idlistwseg['customer_id_hashed'].nunique())


(32396452, 2) 32396452


In [19]:
pd.read_csv('./Q3_mapping_file/BL_mapping_file_2020Q3_JL_2020-11-24.csv',nrows=10)

,customer_id_hashed,email_address_hash,segment
0,bbb02589dc717718c03480d9424d8450f33a6e158072e5...,f11f2329b2b066b9a039931c0f1963ed8ff221c352be72...,C_Legacy_H_Active_N
1,7c058afa0a6e15371f0b27c39228ff3b5bc25dd1c46174...,d2c663a798bfb99e05b0845c6285390f6e04e20de2ac81...,C_Legacy_H_Active_N
2,270071128fc233e866c1990f0528d6f8583afee65675c7...,2eaf012beaf55622f3378708e2badcdc7ee5e73f72d8c3...,C_Legacy_H_Active_N
3,669c72b64c5068f1a9c3b64a6faf9b2052025264ec4b0d...,b662fdeae7d8558587123e915573234da967ffde456210...,C_Legacy_H_Active_N
4,4d6fb5ab42bfe61a00618a6ce811a2d1214a32f78c94f4...,58869f80a66136f4531136259dbb9ac6ddc0925d7fa27c...,C_Legacy_H_Active_N
5,5abc69fd6047c618367245164b8596b3aaf47ceef003f6...,9663d00f0d0126c9ec62daa344165fa7730fa0118a63e3...,C_Legacy_H_Active_N
6,1ea15eef4a77d5627e416f6419ee96a2cd3f3c10ddf410...,da4a0aba28f345553725a7517fcea4064c8a5c3e518fbe...,C_Legacy_H_Active_N
7,7c8ff9aed48bf562fec04e39b1934ccc7a4234ab6f5823...,e429706c833beae9c4145156119bcc450d15c8ef39cfa5...,C_Legacy_H_Active_N
8,d073e952c77cbb93571325ab05deedb922b17a6e7bc2a3...,dafb7a5b423fc2b0a8e151c10613050ea696a600d4300a...,C_Legacy_H_Active_N
9,632d75b0c2dacc4c0939c9895d9537f3806e336d2269d3...,77df3ee6d6714f5370bf0f39b5ba22c85e550b0fe6db4e...,C_Legacy_H_Active_N


In [18]:
idlistwseg.head(2)

,customer_id_hashed,segment
0,bbb02589dc717718c03480d9424d8450f33a6e158072e5...,C_Legacy_H_Active_N
1,7c058afa0a6e15371f0b27c39228ff3b5bc25dd1c46174...,C_Legacy_H_Active_N


In [21]:
list_seg=idlistwseg['segment'].unique().tolist()
list_seg.sort()
list_seg

['C_Legacy_H_Active_N',
 'C_Legacy_H_Active_Y',
 'C_Legacy_L_Active_N',
 'C_Legacy_L_Active_Y',
 'C_Legacy_L_Lapsed1318_N',
 'C_Legacy_L_Lapsed1318_Y',
 'C_Legacy_M_Active_N',
 'C_Legacy_M_Active_Y',
 'C_Legacy_NA_Lapsed1948_N',
 'C_Legacy_NA_Lapsed1948_Y',
 'C_SOTF_H_Active_N',
 'C_SOTF_H_Active_Y',
 'C_SOTF_L_Active_N',
 'C_SOTF_L_Active_Y',
 'C_SOTF_L_Lapsed1318_N',
 'C_SOTF_L_Lapsed1318_Y',
 'C_SOTF_M_Active_N',
 'C_SOTF_M_Active_Y',
 'C_SOTF_NA_Lapsed1948_N',
 'C_SOTF_NA_Lapsed1948_Y',
 'NewReward_081520_P',
 'NewReward_081520_S',
 'NewReward_081520_T',
 'NewReward_082220_P',
 'NewReward_082220_S',
 'NewReward_082220_T',
 'NewReward_090520_P',
 'NewReward_090520_S',
 'NewReward_090520_T',
 'NewReward_091920_P',
 'NewReward_091920_S',
 'NewReward_091920_T',
 'NewReward_100320_P',
 'NewReward_100320_S',
 'NewReward_100320_T',
 'NewReward_101720_P',
 'NewReward_101720_S',
 'NewReward_101720_T',
 'T_Legacy_H_Active_N',
 'T_Legacy_H_Active_Y',
 'T_Legacy_L_Active_N',
 'T_Legacy_L_Activ

In [22]:
list_seg_include=[x for x in list_seg if x[:2] in ['T_',"C_"] and "1948" not in x]
list_seg_include

['C_Legacy_H_Active_N',
 'C_Legacy_H_Active_Y',
 'C_Legacy_L_Active_N',
 'C_Legacy_L_Active_Y',
 'C_Legacy_L_Lapsed1318_N',
 'C_Legacy_L_Lapsed1318_Y',
 'C_Legacy_M_Active_N',
 'C_Legacy_M_Active_Y',
 'C_SOTF_H_Active_N',
 'C_SOTF_H_Active_Y',
 'C_SOTF_L_Active_N',
 'C_SOTF_L_Active_Y',
 'C_SOTF_L_Lapsed1318_N',
 'C_SOTF_L_Lapsed1318_Y',
 'C_SOTF_M_Active_N',
 'C_SOTF_M_Active_Y',
 'T_Legacy_H_Active_N',
 'T_Legacy_H_Active_Y',
 'T_Legacy_L_Active_N',
 'T_Legacy_L_Active_Y',
 'T_Legacy_L_Lapsed1318_N',
 'T_Legacy_L_Lapsed1318_Y',
 'T_Legacy_M_Active_N',
 'T_Legacy_M_Active_Y',
 'T_SOTF_H_Active_N',
 'T_SOTF_H_Active_Y',
 'T_SOTF_L_Active_N',
 'T_SOTF_L_Active_Y',
 'T_SOTF_L_Lapsed1318_N',
 'T_SOTF_L_Lapsed1318_Y',
 'T_SOTF_M_Active_N',
 'T_SOTF_M_Active_Y']

In [23]:
# Use 2020 Q3 regular campaing only
idlistwseg=idlistwseg[idlistwseg['segment'].isin(list_seg_include)]

print(idlistwseg.shape,idlistwseg['segment'].nunique())
print(idlistwseg['customer_id_hashed'].nunique())
gc.collect()

(20915869, 2) 32
20915869


NameError: name 'gc' is not defined

In [25]:
gc.collect()

108

In [26]:
df_qc=idlistwseg.groupby("segment")['customer_id_hashed'].count().to_frame().reset_index()
df_qc['seg_base_name']=df_qc['segment'].apply(lambda x: x[2:])
df_qc['group']=df_qc['segment'].apply(lambda x: x[0])
df_qc=df_qc.pivot(index="seg_base_name",columns="group",values="customer_id_hashed")
df_qc.head(2)

group,C,T
seg_base_name,,
Legacy_H_Active_N,17491,714213
Legacy_H_Active_Y,111841,4566738


In [27]:
df_qc['c_pctg']=df_qc['C']/(df_qc['T']+df_qc['C'])
df_qc

group,C,T,c_pctg
seg_base_name,,,
Legacy_H_Active_N,17491,714213,0.023904
Legacy_H_Active_Y,111841,4566738,0.023905
Legacy_L_Active_N,8257,337171,0.023904
Legacy_L_Active_Y,43532,1777547,0.023905
Legacy_L_Lapsed1318_N,8174,333765,0.023905
Legacy_L_Lapsed1318_Y,40532,1655051,0.023904
Legacy_M_Active_N,14896,608266,0.023904
Legacy_M_Active_Y,83463,3407990,0.023905
SOTF_H_Active_N,9622,392898,0.023904


In [28]:
list_seg=idlistwseg['segment'].unique().tolist()
list_seg.sort()
list_t=[x for x in list_seg if x[0]=="T"]
list_c=[x for x in list_seg if x[0]=="C"]
print(len(list_t))
print(len(list_c))

list_extra_control=[x for x in list_c if x[2:] not in [x[2:] for x in list_t]]
if len(list_extra_control)>0:
    print("remove extra segments in control: %s"%list_extra_control)
idlistwseg=idlistwseg[~idlistwseg['segment'].isin(list_extra_control)]

16
16


In [29]:
df_checking=idlistwseg.groupby('segment')['customer_id_hashed'].count().to_frame().reset_index()
df_checking['T_C_group']=df_checking['segment'].str[0]
df_checking['segment_name']=df_checking['segment'].str[2:]

df_checking=df_checking.pivot_table(index="segment_name",columns='T_C_group',values="customer_id_hashed")
df_checking['pctg']=df_checking['T']/(df_checking['T']+df_checking['C'])


In [30]:
df_checking['pctg'].min(),df_checking['pctg'].max()

(0.9760950526900978, 0.976100067009158)

In [31]:
seg_list=sorted(idlistwseg['segment'].unique().tolist())
len(seg_list)

32

In [32]:
print(idlist.shape)
idlist=pd.merge(idlist,idlistwseg,on="customer_id_hashed",how="left")

idlist=idlist[pd.notnull(idlist['segment'])]
print(idlist.shape)
idlist.head(2)

(31150779, 49)


KeyboardInterrupt: 

In [ ]:
print(idlist.shape,idlist['customer_id_hashed'].nunique(),idlist['Customer_Link'].nunique())

In [ ]:
print(datetime.datetime.now())

In [ ]:
idlist=idlist.drop_duplicates()

idlist[idlist['Customer_Link']=="UNMATCHED"]

In [ ]:
gc.collect()

In [23]:
list_control_customer_ids = idlistwseg[idlistwseg['segment'].str.contains('C_')]['customer_id_hashed'].unique().tolist()
print(len(list_control_customer_ids))
## 
idlistwseg_c2 = idlistwseg[idlistwseg['segment'].str.contains('C_')]
idlistwseg_c2 = idlistwseg_c2.drop_duplicates('customer_id_hashed')
print(len(idlistwseg.index))
idlistwseg = idlistwseg[~idlistwseg['customer_id_hashed'].isin(list_control_customer_ids)]
print(len(idlistwseg.index))

499985
20915869
20415884


In [24]:
idlistwseg = idlistwseg.append(idlistwseg_c2,ignore_index = True)
# print(len(idlistwseg.index))

In [25]:
idlist['Test'] = np.where((idlist['customer_id_hashed'].isin(list_control_customer_ids)),
                          'C','T')
idlist['Test'] = np.where(((idlist['impression']>0)|(idlist['click']>0)),
                          idlist['Test'] + 'E',idlist['Test'] + 'N')
print (idlist['Test'].unique())

['TN' 'CN' 'TE' 'CE']


In [26]:
test = idlist[idlist['Test']=='TE']
test = test.sort_values(['total_transaction_amt','total_transactions'],ascending = False) # 4000 selected
# test = test.sort_values(['total_transactions','total_transaction_amt'],ascending = False) # 100 selected
test = test[['Customer_Link','total_transaction_amt','total_transactions']]
test.head(30)


,Customer_Link,total_transaction_amt,total_transactions
3472378,XY1468uEu3wWcmW5bYguDfJ3jiFcadmclhccCbUA6w604dKJo,14275.23,43.0
10281090,XY1468muMsQ0kDoONh-phCKIzw51A_xalhyMSiyrw1nmhERLc,11322.71,386.0
8243737,XY1468fHWhQk4L1JAllYoz6OZ-cQfiO11VRUrc8nzIwh7CzoQ,10652.47,83.0
14376249,XY14686P6efr4ePOM-T7OFwMDepP2b1GDuw07txX_MvYXh_Kc,8686.89,332.0
28759227,XY1468Wenx4D-Bk8r4FmuI4DuPP7j1vxFi1tOperkK1tmT7s4,7383.40,38.0
23527594,XY1468y_wkrfqYj9a91ma_xYGiWpHiIYOYMRbgMBlSSFezDZY,7124.92,52.0
7488142,XY1468k_8OndGXf1BLv4RCUhOvN8SuVTFfUtGGIweEoR1u7-8,6616.39,125.0
9130870,XY1468bWsC55P8KTg766X3UU0Vssq1QCjgVJ8_XiQMIgouQ78,6243.85,3.0
13951241,Xi1468MuH-Frmmp09h0kKFXCwukjr7rCtWPqrbI1d9AP02...,5712.79,5.0
7096651,XY1468Sa6USGAufo1VjDSvTZ0qaM-d3-Gc11DozvmoiIWge0A,5608.58,32.0


In [27]:
test = idlist[idlist['Test']=='TE']
# test = test.sort_values(['total_transaction_amt','total_transactions'],ascending = False) # 4000 selected
test = test.sort_values(['total_transactions','total_transaction_amt'],ascending = False) # 100 selected
test = test[['Customer_Link','total_transaction_amt','total_transactions']]
test.head(20)

,Customer_Link,total_transaction_amt,total_transactions
10281090,XY1468muMsQ0kDoONh-phCKIzw51A_xalhyMSiyrw1nmhERLc,11322.71,386.0
14376249,XY14686P6efr4ePOM-T7OFwMDepP2b1GDuw07txX_MvYXh_Kc,8686.89,332.0
25576631,XY1468HgqrI7aA8hxlaan4DzKLW4zsgHPDhPOZxhpIoD7ZyYQ,5355.04,305.0
13640488,XY1468_QtwneSZZIhqYrOt-vPcODr4S85iF37tjsFWl9MmkqI,5503.18,200.0
15125770,XY14686boITCsCEy0CDJbkyy28aNreYhGNqqVZ1kAqW09hwqw,4609.23,175.0
24464454,XY146835txiKidZe9pjx5ayGAEpp-5bsTUEbWUc2x0G2Lwr5A,3576.08,174.0
18204548,XY1468KBlZu_iKTtvPYXSm1W39mvIFKe1Cz4dMrPNdVm_IA60,2571.35,143.0
11582129,XY1468KDNpDjH-FHVGUv0DRBMVY_quR2ihKQzPBGod-1d65zc,4626.77,130.0
26388009,Xi1468D87H5dYlw7lgLHk46_xmzkiF868doaBjDt-GYTg7...,1630.61,126.0
7488142,XY1468k_8OndGXf1BLv4RCUhOvN8SuVTFfUtGGIweEoR1u7-8,6616.39,125.0


In [28]:
# 
test = idlist[((idlist['total_transaction_amt']>=4000)|(idlist['total_transactions']>=100))]
print(test.shape)
print(idlist.shape)

test = test[['Customer_Link','customer_id_hashed','Test']]
test = pd.merge(test,idlistwseg,on = 'customer_id_hashed')
test = test.drop_duplicates('customer_id_hashed')
print(test.shape)

print("%d ids excluded from the whole pool, with many impr/click records"%test.shape[0])

(1411, 51)
(20126319, 51)
(1411, 4)
1411 ids excluded from the whole pool, with many impr/click records


In [29]:
print (len(idlist.index))
idlist = idlist[((idlist['total_transaction_amt']<4000) & (idlist['total_transactions']<100))]
print (len(idlist.index))

20126319
20124908


In [30]:
idlist.head(2)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_M_Active_Y,TN
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TN


In [31]:
# When the original design didn't consider the email subscription, use the sub file to add the label
# For Q3, it's already in the segment name

'''
idlist['subscription']=np.where(idlist['segment'].str[-1]=="N","unsubscribed","default_subscribed")
'''


# Changed back per BR's request

list_unsunsribe_ids=pd.read_csv("./BL_Email_UnSubscriber_File_Refresh__20201025040442.csv",
                         dtype=str,usecols=['customersummary_c_primaryscnhash'])['customersummary_c_primaryscnhash'].unique().tolist()
print("len(list_unsunsribe_ids)",len(list_unsunsribe_ids))

idlist['subscription']=np.where(idlist['customer_id_hashed'].isin(list_unsunsribe_ids),"unsubscribed","default_subscribed")
del list_unsunsribe_ids
gc.collect()



len(list_unsunsribe_ids) 6767525


134

In [32]:
idlist.head(2)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test,subscription
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_M_Active_Y,TN,default_subscribed
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TN,default_subscribed


In [33]:
idlist.shape,idlist['customer_id_hashed'].nunique()

((20124908, 52), 20124908)

In [34]:
idlist['subscription'].unique()

array(['default_subscribed', 'unsubscribed'], dtype=object)

In [35]:
idlist.groupby(["subscription"])['customer_id_hashed'].count().to_frame().reset_index()

,subscription,customer_id_hashed
0,default_subscribed,16945989
1,unsubscribed,3178919


In [36]:
# unsub last q
2907860/(2907860+14697375)

0.1651701894351311

In [37]:
# unsub this q
3016650/(3016650+17108258)

0.14989633741431266

In [38]:
idlist.head(3)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test,subscription
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_M_Active_Y,TN,default_subscribed
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TN,default_subscribed
2,XY1468rkWPdZ2FDvKuzE3YF-v5h1RKSfx5NElI5D9gqiIOPwc,a29988754c81f8cbef6c7e267b5d59adf372254880ad06...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_SOTF_L_Lapsed1318_Y,TN,default_subscribed


In [39]:
print(datetime.datetime.now())
# updated the col list below

2020-12-17 16:20:26.183661


In [40]:
list_sales_cols=idlist.columns.tolist()

list_sales_cols=list_sales_cols[list_sales_cols.index("online_transactions_units"):list_sales_cols.index("Other_Furniture_buyers")+1]
list_sales_cols[-1]


'Other_Furniture_buyers'

In [41]:
idlist.head(2)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test,subscription
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_M_Active_Y,TN,default_subscribed
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TN,default_subscribed


In [42]:
dfsum1 = idlist.sort_values(['Test','subscription'],ascending = False)
dfsum1 = dfsum1.drop_duplicates('customer_id_hashed')
print(dfsum1.shape)
dfsum1 = dfsum1.groupby(['Test','subscription']).sum()
dfsum1.reset_index(inplace = True)

dfsum2 = dfsum1.copy()
for i in list_sales_cols+['impression', 'click', 'activity', 
          'impressionunique','clickunique','activityunique',
          'IDs']:
    dfsum2[i] = dfsum2[i]/dfsum2['IDs']*1000000

(20124908, 52)


In [43]:
dfsum1 = idlist.sort_values(['Test','subscription'],ascending = False)
dfsum1 = dfsum1.drop_duplicates('customer_id_hashed')
print(dfsum1.shape)
dfsum1 = dfsum1.groupby(['Test','subscription']).sum()
dfsum1.reset_index(inplace = True)

dfsum2 = dfsum1.copy()
for i in list_sales_cols+['impression', 'click', 'activity', 
          'impressionunique','clickunique','activityunique',
          'IDs']:
    dfsum2[i] = dfsum2[i]/dfsum2['IDs']*1000000

dfsum3 = dfsum1.copy()
for i in list_sales_cols+['impression', 'click', 'activity', 
          'impressionunique','clickunique','activityunique']:
    dfsum3[i] = dfsum3[i]/dfsum3['IDs']
    
dfsum1 = dfsum1.append(dfsum2,ignore_index = True)
dfsum1 = dfsum1.append(dfsum3,ignore_index = True)  
dfsum1 = dfsum1[['Test','subscription','IDs','impression', 'impressionunique','click','clickunique',
                 'activity', 'activityunique']+list_sales_cols]
dfsum1.to_csv(outputpath + 'summary1_by_test_group_12_rows_summary1_15_2_subscribe.csv',index = False)

(20124908, 52)


In [44]:
idlist.head(2)

,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test,subscription
0,Xi14684bNzzUHbs_odkVrPvdY74T9lRQ-TTAjeCcqCkQEJ...,776f396f3588dd38c559c71a1945f23e6cca7c8f6b7d4a...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_M_Active_Y,TN,default_subscribed
1,XY14684O5A3g-3oj_Mnmv8Ckp-hwpoQ2wNMBODyMpbeWruUgw,752bab34bbde7daf9cdd3a207712c9c7b465ea514eb6b0...,0.0,0.0,0.0,5.0,1.0,50.3,5.0,1.0,50.3,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TN,default_subscribed


In [45]:
dfsum1 = idlist.copy()
dfsum1 = dfsum1.drop_duplicates(['customer_id_hashed','segment'])
dfsum1 = dfsum1.groupby(['segment','Test','subscription']).sum()
dfsum1.reset_index(inplace = True)
dfsum2 = dfsum1.copy()

for i in list_sales_cols+['impression', 'click', 'activity', 
          'impressionunique','clickunique','activityunique',
          'IDs']:
    dfsum2[i] = dfsum2[i]/dfsum2['IDs']*1000000
    
dfsum1 = dfsum1.append(dfsum2,ignore_index = True)  
dfsum1 = dfsum1[['Test','segment','subscription','IDs','impression', 'impressionunique','click','clickunique',
                 'activity', 'activityunique']+list_sales_cols]
dfsum1.to_csv(outputpath + 'summary2_by_each_segments_many_rows_summary2_15_2_subscribe.csv',index = False)


In [46]:
site_mapping_folder="/mnt/drv5/dcm_logs/match_tables/"

def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)
            
mapping_files=list(recursive_file_gen(site_mapping_folder))

site_files=[x for x in mapping_files if "sites" in x]
df_sites=pd.DataFrame({"file_path":site_files})
df_sites['start_date']=df_sites['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[5])) # int
df_sites['end_date']=df_sites['file_path'].apply(lambda x: int(os.path.basename(x).split("_")[6]))

df_sites=df_sites[((df_sites['end_date']>=int(startdate.replace("-",""))) & (df_sites['end_date']<=int(enddate.replace("-",""))))]

sites_data_df=pd.DataFrame()
for site_file in df_sites['file_path'].tolist():
    df=pd.read_csv(site_file,dtype=str,compression="gzip")
    sites_data_df=sites_data_df.append(df)


In [47]:
df_map=sites_data_df[['Site ID (DCM)','Site (DCM)']].drop_duplicates('Site ID (DCM)')
# Fetch the site id name the same as placement id without any filter
df_map['Site ID (DCM)']=df_map['Site ID (DCM)'].astype(int)
df_map.head(2)

,Site ID (DCM),Site (DCM)
0,1232325,3interactive.net
1,1232521,AOL - Advertising


In [48]:
df_map.shape

(791, 2)

In [49]:
idteste = idlist[idlist['Test']=='TE']

In [50]:
###impressions
df_imp = pd.read_csv(outputpath + 'data_impression.csv')

df_imp['impression'] = 1

df_imp = pd.merge(df_imp,df_map,on = 'Site ID (DCM)',how="left")
df_imp = df_imp[['Customer_Link','Site (DCM)','Event Time']].groupby(['Customer_Link',
                         'Site (DCM)']).count().unstack('Site (DCM)')
df_imp.columns = df_imp.columns.get_level_values(1)
cols = []
for i in df_imp.columns:
    cols = cols + ['imp_'+i]
df_imp.columns = cols
df_imp.reset_index(inplace = True)
df_imp.fillna(0,inplace = True)
idteste = pd.merge(idteste,df_imp,on = 'Customer_Link',how='left')

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [51]:
####clicks
df_click = pd.read_csv(outputpath + 'data_click.csv')

df_click = pd.merge(df_click,df_map,on = 'Site ID (DCM)',how="left")
df_click = df_click[['Customer_Link','Site (DCM)','Event Time']].groupby(['Customer_Link',
                         'Site (DCM)']).count().unstack('Site (DCM)')
df_click.columns = df_click.columns.get_level_values(1)
cols = []
for i in df_click.columns:
    cols = cols + ['click_'+i]
df_click.columns = cols
df_click.reset_index(inplace = True)
df_click.fillna(0,inplace = True)
idteste = pd.merge(idteste,df_click,on = 'Customer_Link',how='left')

In [52]:
idteste['totalimpclick'] =  idteste['impression'] + idteste['click']

idteste = idteste.drop_duplicates('customer_id_hashed')

idteste = idteste.sort_values(['totalimpclick','click','impression'],
                             ascending = False)
idteste.reset_index(drop = True, inplace = True)
idteste.reset_index(inplace = True)
idteste['index'] = np.ceil((idteste['index']+1)*10/len(idteste.index))
idteste.fillna(0,inplace = True)

In [53]:
idteste.to_csv(outputpath + 'idexposedetail_subscribe.csv')

In [3]:
df_see=pd.read_csv("./output_regular_only_2020-12-05/idexposedetail_subscribe.csv")
df_see.shape

(685976, 62)

In [5]:
df_see.head(2)

,Unnamed: 0,index,Customer_Link,customer_id_hashed,online_transactions_units,online_transactions,online_revenue,instore_transactions_units,instore_transactions,instore_revenue,total_transaction_units,total_transactions,total_transaction_amt,unique_buyers,div9_online_units,div9_online_trans,div9_online_revenue,div9_instore_units,div9_instore_trans,div9_instore_revenue,div9_units,div9_transactions,div9_sales,div9_buyers,Matress_Furniture_online_units,Matress_Furniture_online_trans,Matress_Furniture_online_revenue,Matress_Furniture_instore_units,Matress_Furniture_instore_trans,Matress_Furniture_instore_revenue,Matress_Furniture_units,Matress_Furniture_transactions,Matress_Furniture_sales,Matress_Furniture_buyers,Other_Furniture_online_units,Other_Furniture_online_trans,Other_Furniture_online_revenue,Other_Furniture_instore_units,Other_Furniture_instore_trans,Other_Furniture_instore_revenue,Other_Furniture_units,Other_Furniture_transactions,Other_Furniture_sales,Other_Furniture_buyers,impression,impressionunique,click,clickunique,activity,activityunique,IDs,segment,Test,subscription,imp_The Trade Desk,imp_YouTube.com,click_Facebook,"click_Facebook Inc – Instagram, LLC",click_Pinterest Inc.,click_The Trade Desk,click_YouTube.com,totalimpclick
0,0,1.0,XY1468DXVTXFzDAkniT1ZL1OCqdOAmN3NP2T6x-D-FLgaDaaM,f390fc29a3613033692a68351dbecd81afe8d54c1ec9c5...,0.0,0.0,0.0,184.0,10.0,508.9,184.0,10.0,508.9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,1245.0,1.0,0.0,0.0,0.0,0.0,1,T_Legacy_H_Active_Y,TE,default_subscribed,1245.0,0.0,0.0,0.0,0.0,0.0,0.0,1245.0
1,1,1.0,XY1468vbaIynfItlGfDWLU90oaGT8wLAbw8GYMvuYNIJqERd4,302e133eb879aaf29d6cb175749df0e643d6befa4e155b...,0.0,0.0,0.0,14.0,1.0,58.7,14.0,1.0,58.7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,511.0,1.0,0.0,0.0,0.0,0.0,1,T_SOTF_H_Active_Y,TE,default_subscribed,511.0,0.0,0.0,0.0,0.0,0.0,0.0,511.0


In [54]:
idteste_sum = idteste.groupby('index').sum()
idteste_sum.to_csv(outputpath + 'exposedsum_subscribe.csv')
idteste_median = idteste.groupby('index').median()
idteste_median.to_csv(outputpath + 'exposedmedian_subscribe.csv')

In [55]:
idteste_sub=idteste[idteste['subscription']=="default_subscribed"]
idteste_unsub=idteste[idteste['subscription']=="unsubscribed"]
# sub
idteste_sub = idteste_sub.drop_duplicates('customer_id_hashed')

idteste_sub = idteste_sub.sort_values(['totalimpclick','click','impression'],
                             ascending = False)

if "index" in idteste_sub.columns.tolist():
    del idteste_sub['index']
if "level_0" in idteste_sub.columns.tolist():
    del idteste_sub['level_0']
    
idteste_sub=idteste_sub.reset_index()
del idteste_sub['index']
idteste_sub=idteste_sub.reset_index()

idteste_sub['index'] = np.ceil((idteste_sub['index']+1)*10/len(idteste_sub.index))
idteste_sub.fillna(0,inplace = True)

# unsub
idteste_unsub = idteste_unsub.drop_duplicates('customer_id_hashed')

idteste_unsub = idteste_unsub.sort_values(['totalimpclick','click','impression'],
                             ascending = False)
if "index" in idteste_unsub.columns.tolist():
    del idteste_unsub['index']
if "level_0" in idteste_unsub.columns.tolist():
    del idteste_unsub['level_0']
    
idteste_unsub=idteste_unsub.reset_index()
del idteste_unsub['index']
idteste_unsub=idteste_unsub.reset_index()

idteste_unsub['index'] = np.ceil((idteste_unsub['index']+1)*10/len(idteste_unsub.index))
idteste_unsub.fillna(0,inplace = True)


idteste_sub_label=idteste_sub.append(idteste_unsub)
idteste_sum = idteste_sub_label.groupby(['subscription','index']).sum()
idteste_sum.to_csv(outputpath + 'exposedsum_subscribe.csv')
idteste_median = idteste_sub_label.groupby(['subscription','index']).median()
idteste_median.to_csv(outputpath + 'exposedmedian_subscribe.csv')

In [56]:
idlist_c = idlist[idlist['Test']=='CN']
idlist_c = idlist_c[['customer_id_hashed']].drop_duplicates()
# print len(idlist_c.index)
idlist_c.to_csv(outputpath + 'notexposed_control_idlist.csv',index = False)

In [57]:
idlist_c = idlist[idlist['Test']=='TE']
# print sum(idlist_c['total_transaction_amt'])
idlist_c = idlist_c[['customer_id_hashed','Customer_Link']].drop_duplicates()
# print len(idlist_c.index)
idlist_c.to_csv(outputpath + 'exposed_test_idlist.csv',index = False)

In [58]:
idteste[idteste['click']>0].shape

(47074, 61)

In [59]:
idteste['click'].sum()

69390.0